### Vehicle detection

In [3]:
import matplotlib.image as mpimg
from scipy.ndimage.measurements import label
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import tree
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from moviepy.editor import VideoFileClip
%matplotlib inline

#### color conversion function

The use of this function is to convert the image sent into the colorspace specified by the user

usage :
    
    
    conv_img = convert_color(original_img , 'RGB2YCrCb')

In [4]:
def convert_color(img,conv = 'RGB2YCrCb'):
    
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    
    if conv == 'RGB2HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    else:
        return img
    

#### histogram of gradient function

The use of this function is to find the histogram of the gradience in a single dimentional image send into the function . 
along with the image , we also send in the no of orientations , pix per cell , cells per block etc ... 

usage :
    
    
    hog = (img,9,8,2,vis = False ,feature_vec = True)

In [5]:
def get_hog_features (img,orient,pix_per_cell,cell_per_block,vis = False ,feature_vec = True):
    
    if vis == False:
        feature = hog(img , orientations = orient ,
                     pixels_per_cell = (pix_per_cell,pix_per_cell),
                     cells_per_block = (cell_per_block,cell_per_block),
                     transform_sqrt=True, 
                    visualise=vis, feature_vector=feature_vec)
        return feature
    
    else:
        feature , fimg =  hog(img , orientations = orient ,
                             pixels_per_cell = (pix_per_cell,pix_per_cell),
                             cells_per_block = (cell_per_block,cell_per_block),
                             transform_sqrt=True, 
                             visualise=vis, feature_vector=feature_vec)
        return feature , fimg

#### color bin function

The use of this function is to return a single dimention vector of raw pixel value after resizing the image to the specified size . 

usage :
    
    
    bin_feat = bin_spatial(img,(16,16))

In [6]:
def bin_spatial(img,shape = (32,32)):
    
    color1 = cv2.resize(img[:,:,0],shape).ravel()
    color2 = cv2.resize(img[:,:,1],shape).ravel()
    color3 = cv2.resize(img[:,:,2],shape).ravel()
    
    return np.concatenate((color1,color2,color3))


#### color histogram function

The use of this function is to return a single dimention vector of color histogram of the image . it takes in the image and no of sub divisions (nbins) . 

usage :
    
    
    histogram_of_color = color_hist(img , nbins = 16)

In [7]:
def color_hist(img , nbins = 32):
    
    color1 = np.histogram(img[:,:,0],bins = nbins)
    color2 = np.histogram(img[:,:,1],bins = nbins)
    color3 = np.histogram(img[:,:,2],bins = nbins)
    hist_feat = np.concatenate((color1[0],color2[0],color3[0]))
    
    return hist_feat
    

#### Extract feature function

A unified function to return all the above stated features a a single feature vector for easy usage . 


usage :
    
    feature = extractFeatures (img ,9,8,2,'RGB2YCrCb',vis = False ,feature_vec = True ,nbins = 16 ,shape = (32,32) )
    
    

In [8]:
def extractFeatures (img ,orient,pix_per_cell,cell_per_block,cspace = 'RGB2YCrCb',vis = False ,feature_vec = True ,nbins = 32 ,shape = (32,32) ):
    
    cvtimg = convert_color(img,cspace)
    color_hist_feat = color_hist(cvtimg , nbins)
    hog_feat1 = get_hog_features (cvtimg[:,:,0],orient,pix_per_cell,cell_per_block,vis = False ,feature_vec = True)
    hog_feat2 = get_hog_features (cvtimg[:,:,1],orient,pix_per_cell,cell_per_block,vis = False ,feature_vec = True)
    hog_feat3 = get_hog_features (cvtimg[:,:,2],orient,pix_per_cell,cell_per_block,vis = False ,feature_vec = True)  
    
    return np.concatenate(( color_hist_feat , hog_feat1, hog_feat2, hog_feat3))
    
    

#### bounding box draw function

function to find the no of cars in the heat map and draw a bounding box . 

usage opt_img = drawHeatMap(img , labels , xstart)

    

In [9]:
def drawHeatMap(img , labels , xstart):
    
    for car_number in range(1,labels[1]+1): # no of car detected
        
        nonzero = (labels[0] == car_number).nonzero()
        nonzerox = np.array(nonzero[1])
        nonzeroy = np.array(nonzero[0])
        bbox = ((np.min(nonzerox)+xstart,np.min(nonzeroy)),(np.max(nonzerox)+xstart,np.max(nonzeroy)))
        img = cv2.rectangle(img,bbox[0],bbox[1],(0,0,255),6)
        
    return img

#### find car function


steps done in this fuction:

* resize the image 
* find the scan window size based on the input scale values
* find the HOG of the image
* slide the window throught the image area specified by x , y start and end points to obtain search image
* find the feature vector 
* pass the feature vector throught the classifier and predict
* based on the prediction , mark the window where the predictions are equal to 1
* generate heat map based on prediction 
* draw the bounding box using the heat map to box function
* return the final image with cars labled

In [10]:
def find_cars(img, ystart, ystop,xstart , xstop, scaleList, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    box = []
    img = img.astype(np.float32)/255
    
    for scale in scaleList:
        
        img_tosearch = img[ystart:ystop,xstart:xstop,:]
        ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
        
        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]
        # Define blocks and steps as above
        nxblocks = (ch1.shape[1] // pix_per_cell)-1
        nyblocks = (ch1.shape[0] // pix_per_cell)-1 
        nfeat_per_block = orient*cell_per_block**2
        # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
        window = 64
        nblocks_per_window = (window // pix_per_cell)-1 
        cells_per_step = 2  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step
        # Compute individual channel HOG features for the entire image
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

        for xb in range(nxsteps):
            
            for yb in range(nysteps):
                
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                # Extract HOG for this patch
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
                xleft = xpos*pix_per_cell
                ytop = ypos*pix_per_cell
                # Extract the image patch
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
                # Get color features
                spatial_features = bin_spatial(subimg, shape=spatial_size)
                hist_features = color_hist(subimg, nbins=hist_bins)
                # Scale features and make a prediction
                #test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
                test_features = X_scaler.transform(np.hstack((hist_features, hog_features)).reshape(1, -1))     
                test_prediction = svc.predict(test_features)

                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    #cv2.rectangle(draw_img,(xbox_left+xstart, ytop_draw+ystart),(xbox_left+win_draw+xstart,ytop_draw+win_draw+ystart),(0,0,255),6) 
                    box.append ((xbox_left, ytop_draw+ystart,xbox_left+win_draw,ytop_draw+win_draw+ystart))
    
    heatMap = np.zeros_like(draw_img[:,:,0])
    
    for pts in box:
        
        heatMap [pts[1]:pts[3],pts[0]:pts[2]] +=1

    heatMap2 = np.zeros_like(heatMap)
    heatMap2[heatMap>=3] = 1
    labels = label(heatMap2)
    draw_img = drawHeatMap(draw_img , labels , xstart)
                    
    return draw_img

#### get the training images into the environment

Steps done in this code block

* import the training images
* flip the image for data augmentation
* find the features of the images 
* create train and test data with both X and Y (features and lables)


In [11]:
### spcify the path for the training images 
img = glob.glob('D:\\tutorials\\nanodegree\\SDC\\unit_11_Vehicle_Detection\\img\\*.png')
carFar = glob.glob('D:\\tutorials\\nanodegree\\SDC\\unit_11_Vehicle_Detection\\vehicles\\GTI_Far\\*.png')
carLeft = glob.glob('D:\\tutorials\\nanodegree\\SDC\\unit_11_Vehicle_Detection\\vehicles\\GTI_Left\\*.png')
carRight = glob.glob('D:\\tutorials\\nanodegree\\SDC\\unit_11_Vehicle_Detection\\vehicles\\GTI_Right\\*.png')

cars = []
notcars = []
for image in img:
    
    if 'extra' in image:
        
        notcars.append(image)
    else:
        
        cars.append(image)

for image in carFar:
    
    if 'extra' in image:
        
        notcars.append(image)
    else:
        
        cars.append(image)

for image in carLeft:
    
    if 'extra' in image:
        
        notcars.append(image)
        
    else:
        
        cars.append(image)

for image in carRight:
    
    if 'extra' in image:
        
        notcars.append(image)
        
    else:
        
        cars.append(image)

orient = 9
pix_per_cell = 8
cell_per_block = 2

feature_vec = True

carFeat = []
notcarFeat = []

for img in cars:
    
    timg = mpimg.imread(img)
    feature = extractFeatures (timg ,orient,pix_per_cell,cell_per_block,cspace = 'RGB2YCrCb',vis = False ,feature_vec = True ,nbins = 16 ,shape = (32,32) )
    carFeat.append(feature)
    
    #flipping the image for more data
    
    timg = cv2.flip(timg , 1)
    feature = extractFeatures (timg ,orient,pix_per_cell,cell_per_block,cspace = 'RGB2YCrCb',vis = False ,feature_vec = True ,nbins = 16 ,shape = (32,32) )
    carFeat.append(feature)

for img in notcars:
    
    timg = mpimg.imread(img)
    feature = extractFeatures (timg ,orient,pix_per_cell,cell_per_block,cspace = 'RGB2YCrCb',vis = False ,feature_vec = True ,nbins = 16 ,shape = (32,32) )
    notcarFeat.append(feature)
    
X = np.concatenate((carFeat,notcarFeat))
Y = np.concatenate ((np.ones(len(carFeat)).astype(np.int),np.zeros(len(notcarFeat)).astype(np.int)))
x_scalar = StandardScaler().fit(X)
scaled_X = x_scalar.transform(X)
x,y = shuffle(scaled_X , Y)
X_train , X_test , Y_train , Y_test = train_test_split(x, y, test_size=0.2)

#### Train a Linear SVC classifier 

In [12]:
svc = LinearSVC(C=100)
t=time.time()
t2 = time.time()
svc.fit(X_train , Y_train)
print(round(t2-t, 2), 'Seconds to train SVC...')
print(svc.score(X_test, Y_test))

0.0 Seconds to train SVC...
0.997533300444


#### Vehicle Detection function

get an image frame and return the same image frame with box drawn on the vehicles

In [13]:
def vehicle_detection(img):
    
    ystart = 400
    ystop = 656
    xstart = 0
    xstop = 1280
    spatial_size = (32,32)
    hist_bins = 16
    scale = [1.55,1.5,1.4,1.3,0.9,3.0,2.5]#2.2#1.9
    out_img  = find_cars(img, ystart, ystop,xstart,xstop, scale, svc, x_scalar, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    
    return (out_img)

In [ ]:
white_output = 'D:\\tutorials\\nanodegree\\SDC\\unit_11_Vehicle_Detection\\FinalOutput1.mp4'
nameOfTheFile = "D:\\tutorials\\nanodegree\\SDC\\unit_10_Advanced_lane_finding\\CarND-Advanced-Lane-Lines-master\\project_video.mp4"
clip1 = VideoFileClip(nameOfTheFile)
white_clip = clip1.fl_image(vehicle_detection) 
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video D:\tutorials\nanodegree\SDC\unit_11_Vehicle_Detection\FinalOutput1.mp4
[MoviePy] Writing video D:\tutorials\nanodegree\SDC\unit_11_Vehicle_Detection\FinalOutput1.mp4



 22%|████████████████▊                                                          | 282/1261 [1:51:51<2:41:43,  9.91s/it]